In [ ]:
#| default_exp chatgpt

In [ ]:
#| export
import re
from mytools.paths import *
from mytools.tools import *
import openai
openai.api_key = get_api_key(api_path,'openai')

In [ ]:
#| export
from transformers import CLIPProcessor, CLIPModel
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = processor.tokenizer
import pandas as pd

In [ ]:
#| export
def prompt(role,text): return {"role": role, "content": text}

In [ ]:
#| export
def _ask_chatgpt(text,messages=[],prefix=[],role = 'user',return_request=False,model="gpt-3.5-turbo-0301",**kwargs):
    request = prompt(role,text)
    messages = prefix+messages+[request]
    
    return openai.ChatCompletion.create(
                                         model=model,  #"gpt-4", #"gpt-4" "gpt-3.5-turbo" gpt-3.5-turbo-0301
                                         messages=messages,
                                         **kwargs
                                        )

In [ ]:
#| export
def ask_chatgpt(text,messages=[],prefix=[],role = 'user',return_request=False,model="gpt-3.5-turbo-0301",**kwargs):
    request = prompt(role,text)
    r = _ask_chatgpt(text,messages=[],prefix=[],role = 'user',return_request=False,model=model,**kwargs)
    response = r.choices[0].message
    print (response.content)
    if return_request: return response,request
    else: return response
    

In [ ]:
#| export
def ask_chatgpt_stream(text,messages=[],prefix=[],role = 'user',return_request=False,model="gpt-3.5-turbo-0301",**kwargs):
    
    r = _ask_chatgpt(text,messages=messages,prefix=prefix,role = 'user',return_request=False,model=model,stream = True,**kwargs)
    for i in r: 
        try:     yield i.choices[0].delta.content 
        except: pass

In [ ]:
for i in ask_chatgpt_stream('test',model = 'gpt-3.5-turbo-0301'): print(i)

As
 an
 AI
 language
 model
,
 I
 have
 already
 been
 tested
 and
 trained
.
 However
,
 feel
 free
 to
 provide
 me
 with
 specific
 prompts
 or
 questions
 to
 see
 how
 well
 I
 can
 respond
.


In [ ]:
#| export
def chat(text, messages=[],prefix=[],model='gpt-3.5-turbo',append=True,role='user',**kwargs):
    response,request = ask_chatgpt(text,messages=messages,prefix=prefix,role=role,return_request=True,**kwargs)
    if append:
        messages.append(request)
        messages.append(response)
        
    return messages


In [ ]:
#| export
def crop_tokens(messages,max_tokens=4000):
    token_count_df = pd.DataFrame(messages)
    token_count_df['len'] = token_count_df['content'].apply(lambda s:len(s.split(' ')))
    token_count_df = token_count_df[token_count_df['len']<max_tokens]
    token_count_df = token_count_df.sort_index(ascending=False)
    token_count_df['len']=token_count_df['len'].cumsum()
    token_count_df = token_count_df.sort_index()
    token_index = token_count_df[token_count_df['len']<max_tokens].index
    return [messages[i] for i in token_index]

In [ ]:
messages =[prompt('user',f'test{i} '*20) for i in range(2)]

In [ ]:
crop_tokens(messages,1000)

[{'role': 'user',
  'content': 'test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 test0 '},
 {'role': 'user',
  'content': 'test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 test1 '}]

In [ ]:
#| export
def strip_text(text,urls):
    for url in urls: text = text.replace(url,'').strip()
    while True:
        start_len = len(text)
        text = text.replace('  ',' ')
        text = text.replace('\n \n','\n')
        stop_len = len(text)
        if start_len == stop_len: break
        
    return text
    

In [ ]:
#| export
extract_urls = lambda text: re.findall('https?://[^\s]+', text)

get_button_text = lambda url:url.split('q=')[1].split('&')[0].replace('+',' ')

In [ ]:
!nbdev_export